# Description

We'll put a bakery in San Isidro, Buenos Aires. We need to know where is the best place for our bakery. The place is very important because the number of clients we will have depends on it.

# Data

For select the best place we need to know where we have the most clients. To do this we are going to investigate the distribution of bakeries. 

# Methodology

We know that the most bakeries are close to most clients. We check in Foursquare for the location of the bakeries in San Isidro and with this information we calculate and choose the center of the bakeries like the best place for our bakery.

# Import libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


# Configure api

In [2]:
CLIENT_ID = 'VQMGV4XGVVJWY23Y33DIZWWSC03I252BHR0VZC0BGUDDNANK' # your Foursquare ID
CLIENT_SECRET = 'P2LUYORFDKD5I4SBKAEGAGSOO303DZBJVDRQ3EAZWN0WPQ2J' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 10
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VQMGV4XGVVJWY23Y33DIZWWSC03I252BHR0VZC0BGUDDNANK
CLIENT_SECRET:P2LUYORFDKD5I4SBKAEGAGSOO303DZBJVDRQ3EAZWN0WPQ2J


# Configure address

In [3]:
address = 'san isidro, buenos aires, argentina'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-34.4739792 -58.5264866


# Look for another bakeries in the neighbrood

In [4]:
search_query = 'panadería'
radius = 30000

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VQMGV4XGVVJWY23Y33DIZWWSC03I252BHR0VZC0BGUDDNANK&client_secret=P2LUYORFDKD5I4SBKAEGAGSOO303DZBJVDRQ3EAZWN0WPQ2J&ll=-34.4739792,-58.5264866&oauth_token=&v=20180604&query=panadería&radius=30000&limit=10'

# Get the results

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 500,
  'errorType': 'server_error',
  'errorDetail': 'Foursquare servers are experiencing problems. Please retry and check status.foursquare.com for updates.'},
 'response': {}}

# Make the DataFrame

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

KeyError: 'venues'

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

#  Show other bakeries

In [ ]:
dataframe_filtered.name

In [ ]:
venues_map = folium.Map(location=[dataframe_filtered['lat'].mean(), dataframe_filtered['lng'].mean()], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [dataframe_filtered['lat'].mean(),dataframe_filtered['lng'].mean()],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [ ]:
dataframe_filtered['lat'].mean()

In [ ]:
dataframe_filtered['lng'].mean()

# Results

We choose to put our bakery in the center of the others bakeries. In than place we'll have the most customers and in adition, there is no bakeries in this place.

# Conclusion

We get the best place to put a bakery in San Isidro